# Chain

**Chain**(체인)은 여러 컴포넌트(요소)를 정해진 순서대로 연결하여 **복잡한 AI 작업을 단계별로 자동화**할 수 있도록 돕는 구조이다.

- 각 컴포넌트는 입력을 받아 특정 처리를 수행한 후 다음 단계로 결과를 전달한다.
- 복잡한 작업을 여러 개의 단순한 단계로 나누고, 각 단계를 순차적으로 실행함으로써 전체 작업을 체계적으로 구성할 수 있다.

## 기본 개념

- 체인은 하나의 LLM 호출에 그치지 않고 **여러 LLM 호출이나 도구 실행을 순차적으로 연결**할 수 있다.
- 예를 들어, 사용자의 질문 → 검색 → 요약 → 응답 생성 같은 일련의 작업을 체인으로 구성할 수 있다.
- 체인을 사용하면 코드의 재사용성과 유지 보수성이 향상된다.

## LangChain에서의 Chain 구성 방식

LangChain은 다음 두 가지 방식을 통해 체인을 구성할 수 있다.

### 1. Off-the-shelf Chains 방식 (클래식 방식)

- LangChain에서 제공하는 **미리 정의된 Chain 클래스**(예: `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)를 활용하는 방식이다.
- 이 방식은 LangChain의 **초기 구조**이며, 대부분의 클래스는 현재 **더 이상 사용되지 않음(deprecated)** 상태이다.

> 현재 LangChain에서는 이 방식을 권장하지 않는다.

### 2. LCEL (LangChain Expression Language) 방식

- 체인을 함수형 방식으로 선언할 수 있는 **표현식 기반의 체인 구성 언어**이다.
- LCEL 방식은 간결하고 선언적인 문법을 제공하여 **직관적이고 확장성 있는 체인 구성**이 가능하다.
- `Runnable`이라는 공통 인터페이스를 기반으로 다양한 요소를 조합하여 체인을 구성한다.
- 체인의 각 구성 요소는 `invoke()` 메서드로 실행된다.

In [2]:
# off-the-shelf 방식
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt_template = PromptTemplate(
    template="{item}에 어울리는 이름 {count}개를 만들어 주세요."
)
model = ChatOpenAI(model_name="gpt-4o-mini")
parser = StrOutputParser()

# 변수:값 -> (prompt_template) -prompt-> (model) -응답결과-> (parser) -> 최종결과

In [2]:
from langchain import LLMChain
chain = LLMChain(
    prompt=prompt_template,
    llm=model, 
    output_parser=parser
)
response = chain.invoke({"item":"가방", "count":5})

C:\Users\안수민\AppData\Local\Temp\ipykernel_35764\1686607166.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [3]:
response

{'item': '가방',
 'count': 5,
 'text': '물론이죠! 가방에 어울리는 이름 5개를 제안해 드릴게요.\n\n1. **에코 트레블러** - 환경 친화적인 소재로 만든 여행용 가방\n2. **도시의 정수** - 도시에서 필요한 모든 것을 담을 수 있는 멀티 백\n3. **스타일 문** - 시크하고 세련된 디자인의 핸드백\n4. **내가 만든 이야기** - 개인의 개성을 담은 맞춤형 가방\n5. **편안한 동반자** - 일상에서 쉽게 들고 다닐 수 있는 캐주얼 가방\n\n이름들이 가방의 특성이나 디자인에 잘 어울리길 바랍니다!'}

In [4]:
print(response["text"])

물론이죠! 가방에 어울리는 이름 5개를 제안해 드릴게요.

1. **에코 트레블러** - 환경 친화적인 소재로 만든 여행용 가방
2. **도시의 정수** - 도시에서 필요한 모든 것을 담을 수 있는 멀티 백
3. **스타일 문** - 시크하고 세련된 디자인의 핸드백
4. **내가 만든 이야기** - 개인의 개성을 담은 맞춤형 가방
5. **편안한 동반자** - 일상에서 쉽게 들고 다닐 수 있는 캐주얼 가방

이름들이 가방의 특성이나 디자인에 잘 어울리길 바랍니다!


# [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel) (LangChain Expression Language)
- LCEL은 LangChain의 핵심 기능인 체인(Chain)을 더욱 간결하고 유연하게 구성할 수 있도록 고안된 **선언형 체인(chain) 구성 언어**이다.
- 파이프 연산자 `|`를 사용해 선언적 방법으로 여러 작업을 연결한다.
- 체인을 구성하는 각 요소는 [1]`Runnable` 타입이나 [2] 함수 등의 `Callable` 객체로, 체인 내에서 실행 가능한 단위이다.
- 각 단계는 invoke() 메서드를 통해 실행되며, 앞 단계의 출력이 다음 단계의 입력으로 자동 전달된다.
    - [Runnable 컴포넌트별 입출력 타입](https://python.langchain.com/docs/concepts/runnables/#input-and-output-types)
    - 각 컴포넌트의 input과 output 타입에 맞춰 값이 전달되도록 한다.
- https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

참고: Class 객체도 `__call__()` 메소드를 정의하면 Callable이므로 체인 내에서 실행 가능하다.

## [Runnable](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html)
- LangChain의 Runnable은 실행 가능한 작업 단위를 캡슐화한 개념으로, 데이터 흐름의 각 단계를 정의하고 **체인(chain) 에 포함 되어**  복잡한 작업의 각 단계를 수행 한다.
- Chain을 구성하는 class들은 Runnable의 상속 받아 구현한다.
- **Prompt Template클래스**, **Chat 모델, LLM 모델 클래스**, **Output Parser 클래스** 등 다양한 컴포넌트가 Runnable을 상속받아 구현된다.

### 주요 특징
- 작업 단위의 캡슐화:
    - Runnable은 특정 작업(예: 프롬프트 생성, LLM 호출, 출력 파싱 등)을 수행하는 독립적인 컴포넌트이다.
    - 각 컴포넌트는 독립적으로 테스트 및 재사용이 가능하며, 조합하여 복잡한 체인을 구성할 수 있다.
- 체인 연결 및 작업 흐름 관리:
    - Runnable은 체인(chain, 일련의 연결된 작업 흐름)을 구성하는 기본 단위로 사용된다.
    - LangChain Expression Language(LCEL)를 사용하면 | 연산자를 통해 여러 Runnable을 쉽게 연결할 수 있다.
    - 입력과 출력의 형식을 일관되게 유지하여 각 단계가 자연스럽게 연결된다.
- 모듈화 및 디버깅 용이성:
    - 각 단계가 명확히 분리되어 문제 발생 시 어느 단계에서 오류가 발생했는지 쉽게 확인할 수 있다.
    - 복잡한 작업을 작은 단위로 나누어 체계적으로 관리할 수 있다.
      
### Runnable의 표준 메소드
- 모든 Runnable이 구현하는 공통 메소드
    - `invoke()`: 단일 입력을 처리하여 결과를 반환.
    - `batch()`: 여러 입력 데이터들을 한 번에 처리.
    - `stream()`: 입력에 대해 스트리밍 방식으로 응답을 반환.
    - `ainvoke()`: 비동기 방식으로 입력을 처리하여 결과를 반환.

### Runnable의 주요 구현체(하위 클래스)

- `RunnableSequence`
    - 여러 `Runnable`을 순차적으로 연결하여 실행하는 구성이다.
    - 각 단계의 출력이 다음 단계의 입력으로 전달된다.
    - LCEL을 사용하여 체인을 구성할 경우 자동으로 `RunnableSequence`로 변환된다.
-  `RunnablePassthrough`
    - 입력 데이터를 가공하지 않고 그대로 다음 단계로 전달하는 `Runnable`이다.
    - 선택적으로 미리 정의된 키-값 쌍을 함께 전달할 수 있다.

- `RunnableParallel`
    - 여러 `Runnable`을 병렬로 실행한 후, 결과를 결합하여 다음 단계로 전달한다.
    - 병렬 처리를 통해 처리 속도를 개선할 수 있다.

- `RunnableLambda`
    - 일반 함수 또는 `lambda` 함수를 `Runnable`로 변환하여 체인에 포함할 수 있다.
    - 사용자 정의 함수로 동작을 확장할 때 유용하다.

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

#### Runnable 예제

In [6]:
from langchain_core.runnables import Runnable  # 모든 Runnable의 최상위.
# 사용자정의 Runnable
class MyRunnable(Runnable):

    def invoke(self, input_data:str, config:dict=None):
        # invoke(): 구현하는 Runnable이 해야하는 작업을 구현하는 메소드.
        # input_data: 입력값
        # config: 일할 때 필요한 설정값들.
        if config is not None and config.get('lang') == "en":
            return f"Explain {input_data} in one sentences."
        return f"{input_data}에 대해서 한 문장으로 설명해줘."

In [7]:
my_runnable = MyRunnable()
my_runnable.invoke("사과")
my_runnable.invoke("컴퓨터")
my_runnable.invoke("Apple", {"lang":"en"})

'Explain Apple in one sentences.'

In [8]:
from langchain_openai import ChatOpenAI

my_runnable = MyRunnable()
model = ChatOpenAI(model_name="gpt-4o-mini")

prompt = my_runnable.invoke("Apple", {"lang":"en"})
prompt = my_runnable.invoke("Langchain")
response = model.invoke(prompt)
print(response)

content='LangChain은 언어 모델과 외부 리소스를 연결하여 복잡한 애플리케이션을 구축할 수 있도록 도와주는 프레임워크입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 19, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bh3ew2WBnRJ8zNR7bnPgr7pros6DM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--9c3e73c5-044c-45ec-86f7-637507dd8647-0' usage_metadata={'input_tokens': 19, 'output_tokens': 36, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [9]:
# chain -> Runnable | Runnable | Runnable
chain = my_runnable | model
# chain 호출 -> invoke
res = chain.invoke("과일 배")
print(res)

content='과일 배는 부드럽고 달콤한 맛의 과일로, 주로 식용으로 소비되며, 다양한 품종이 존재해 각각의 특징과 맛이 다양합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 20, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bh3f00ASL1TiuYcL5mhEKgIHBGyHW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--6a9960e3-78c8-46a9-b6fe-94503abbaa3c-0' usage_metadata={'input_tokens': 20, 'output_tokens': 42, 'total_tokens': 62, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [10]:
# 기본 체인 구성: prompt_template -> model -> output parser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

# role: system, user/human, ai/assistant
#       system: 채팅  전체에 적용되는 공통 지침을 지정하는 role
prompt_template = ChatPromptTemplate(
    messages=[
        ("system", "당신은 오랜 경력의 한국 관광 가이드입니다. 여행객들에게 설명하듯이 친절하게 답변을 해주세요."),
        ("human", "{query}")
    ]
)
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=1.0)

guide_chain = prompt_template | model | StrOutputParser()

print(type(guide_chain)) # RunnableSequence: Runnable 타입 
#                           ==> chain도 다른 chain의 구성요소로 포함될 수있다.

<class 'langchain_core.runnables.base.RunnableSequence'>


In [11]:
query = "서울에서 꼭 가봐야되는 여행지를 세 곳만 알려줘."
response = guide_chain.invoke({"query":query})

In [12]:
print(response)

서울은 역사와 현대 문화가 어우러진 멋진 도시입니다. 꼭 가봐야 할 여행지를 세 곳 추천해 드릴게요!

1. **경복궁**: 서울의 대표적인 고궁인 경복궁은 조선시대 왕실의 중심지였습니다. 화려한 건축물과 아름다운 정원, 그리고 국립민속박물관까지 함께 방문할 수 있어요. 특히 수문장 교대식은 꼭 보셔야 할 볼거리입니다. 전통의 멋을 느껴보세요!

2. **명동**: 쇼핑과 먹거리의 천국인 명동은 방문객들에게 인기가 많은 지역입니다. 다양한 패션 브랜드는 물론, 길거리 음식도 즐길 수 있어요. 특히 호떡, 떡볶이, 그리고 갓 구운 마카롱 등 맛있는 음식들이 가득하니 놓치지 마세요!

3. **홍대**: 젊은이들의 문화가 살아 있는 홍대는 예술과 음악으로 가득한 공간입니다. 거리 공연과 독특한 카페, 그리고 예술가들의 갤러리를 탐방해 보세요. 밤에는 다양한 클럽과 바에서 활기찬 분위기를 느낄 수 있습니다.

이 세 곳은 서울의 매력을 잘 보여주는 장소들이니 꼭 방문해 보시길 바랍니다!


In [14]:
while True:
    query = input("질문:")
    if query == "!quit":
        break
    resp = guide_chain.invoke({"query":query})
    print("User:", query)
    print("AI:", resp)
    print("-"* 50)

User: 두부로 할 수 있는 요리 알려줘
AI: 안녕하세요! 두부는 매우 다양하게 요리할 수 있는 재료입니다. 여러 가지 요리를 소개해드릴게요.

1. **두부 스테이크**:
   두부를 두툼하게 썰어 팬에 구운 후, 간장과 마늘로 만든 소스를 얹어 드시면 아주 맛있습니다. 채소와 함께 곁들이면 더욱 훌륭한 한 끼가 됩니다.

2. **두부 찌개**:
   순두부찌개나 두부김치찌개는 한국에서 매우 인기 있는 요리입니다. 두부와 돼지고기 또는 해산물, 채소를 넣고 매운 고추장이나 된장을 넣어 끓이면 깊은 맛의 찌개가 완성됩니다.

3. **두부 볶음**:
   두부를 큼직큼직하게 썰어 채소, 고기와 함께 볶아 간장 또는 굴 소스를 첨가하면 간편하면서도 맛있는 반찬이 됩니다.

4. **두부 샐러드**:
   깍둑 썬 두부를 신선한 채소와 함께 드레싱과 곁들여 드시면 맛있고 건강한 샐러드가 됩니다.

5. **두부전**:
   두부에 전분과 다양한 채소를 섞어 반죽한 후 부쳐서 전을 만들면 간단한 간식이나 안주로 좋습니다.

6. **된장 두부무침**:
   부드러운 두부를 으깨고, 된장, 다진 채소, 고추 등을 섞어 무쳐내면 한입에 쏙 넣기 좋은 반찬이 됩니다.

두부는 단백질이 풍부하고 건강에도 좋기 때문에 다양한 방식으로 즐겨보시면 좋아요. 어떤 요리를 시도해보실 건가요? 😊
--------------------------------------------------


#### RunnableLambda 예제

In [15]:
from langchain_core.runnables import RunnableLambda
# RunnableLambda(함수) -> 함수를 실행하는 Runnable을 생성.
my_runnable2 = RunnableLambda(lambda input_data : f"{input_data}를 한 문장으로 설명해줘.")
my_runnable2.invoke("LLM") 

'LLM를 한 문장으로 설명해줘.'

In [16]:
chain = my_runnable2 | model
chain.invoke("LLM")

AIMessage(content='LLM(대형 언어 모델)은 대량의 텍스트 데이터를 기반으로 자연어 처리를 수행하며, 사람의 언어를 이해하고 생성하는 AI 시스템입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 18, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bh4PprrjYCFHC97EEUmLowmeE1Lvy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c6264fe8-3561-48f9-b18a-7c4f817193ee-0', usage_metadata={'input_tokens': 18, 'output_tokens': 38, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
def sum(nums:dict):
    return nums[0] + nums[1]

my_runnable2 = RunnableLambda(sum)
my_runnable2.invoke({0:10, 1:20})

# invoke(입력데이터:str|dict, 설정정보:dict)
## 입력데이터가 여러개일 경우 dict등의 자료구조를 이용해서 받는다.

30

#### RunnablePassthrough 예제

In [13]:
# 1 앞 Runnable이 처리한 결과를 다음 Runnable에 "그대로" 전달.
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")
RunnablePassthrough().invoke({"Key":"value"})

{'Key': 'value'}

In [17]:
# 2. 앞 Runnable이 처리한 결과에 Item을 추가해서 다음 Runnable에 전달.
#   -> 입력으로 dictionary 받아서 거기에 item을 추가.
# RunnablePassthrough.assign(key1=Runnable, key2=Runnable, ...)
#  - 받은 dictionary에 "key1":Runnable반환값, "key2":Runnable반환값, .. 추가해서 다음으로 전달.
address_runnable = RunnableLambda(lambda x: "서울시 금천구")  #"서울시 금천구" 를 반환.
phone_runnable = RunnableLambda(lambda x: "010-1111-2222")

RunnablePassthrough.assign(address=address_runnable, phone=phone_runnable).invoke({"name":"홍길동"})

{'name': '홍길동', 'address': '서울시 금천구', 'phone': '010-1111-2222'}

#### RunnableSequence 예제

In [18]:
from langchain_core.runnables import RunnableSequence

run1 = RunnableLambda(lambda x: x + 1)
run2 = RunnableLambda(lambda x: x * 2)

chain = run1 | run2
print(type(chain))
chain.invoke(30)


<class 'langchain_core.runnables.base.RunnableSequence'>


62

In [22]:
chain2 = RunnableSequence(run1, run2)  #(prompt_template, model,  output_parser)
chain2.invoke(100)

202

#### RunnableParallel 예제

In [23]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

run1 = RunnableLambda(lambda x: x + 1)
run2 = RunnableLambda(lambda x: x * 2)
run3 = RunnableLambda(lambda x: x // 3)

runnable = RunnableParallel(
    {
        "result1":run1,
        "result2":run2,
        "result3":run3,
        "result4":RunnablePassthrough()  # 앞에서 받은 값을 그대로 다음에 전달.
    }
)
# Runnable들을 각각 실행하고 그 결과를 key에 할당한 Dictionary를 반환.
runnable.invoke(20)

{'result1': 21, 'result2': 40, 'result3': 6, 'result4': 20}

In [26]:
run0 = RunnableLambda(lambda x: x + 100)
# run0 -> {run1, run2, run3} -> ..
chain = run0 | {"result1":run1, "result2":run2, "result3":run3}
# 딕셔너리를 LCEL 안에 넣어야 parallel이 된다. 
# LCEL 안에 넣지 않으려면 `RunnableParallel`을 써야 한다.
chain.invoke(20)
# 20을 run0에 넣음 -> 120 도출
# 120을 {}로 묶인 Parallel에 넣는다
# run1, run2, run3에 넣어 처리한 결과를 chain에 넣는다. 

{'result1': 121, 'result2': 240, 'result3': 40}

#### LCEL Chain 예제

In [32]:
# 음식 이름을 받아서 레시피를 출력하는 chain을 구성
# prompt template -> model -> output parser 
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent
prompt_template = PromptTemplate(
    template=dedent("""
    # Instruction
    당신은 숙련된 요리 연구가입니다. 요청한 음식의 레시피를 작성해 주세요.
                    
    # Input data
    음식이름: {food}
    """)
)
model = ChatOpenAI(model_name="gpt-4o-mini")
food_chain = prompt_template | model | StrOutputParser()


In [28]:
response = food_chain.invoke({"food":"pasta"})

In [29]:
print(response)

## 기본 파스타 레시피

### 재료
1. 스파게티 또는 원하는 파스타 200g
2. 소금 1큰술 (물에 넣기 위함)
3. 올리브 오일 2큰술
4. 마늘 3쪽 (다진 것)
5. 양파 1개 (다진 것)
6. 토마토 400g (토마토 소스 또는 신선한 토마토 다진 것)
7. 바질 잎 몇 장 (선택 사항)
8. 파르미지아노 레지아노 치즈 (Grated, 선택 사항)
9. 후추 (맛을 내기 위함)

### 조리 도구
- 큰 냄비
- 팬
- 나무 숟가락 또는 주걱
- 체

### 조리 방법

1. **파스타 삶기**
   - 큰 냄비에 물을 충분히 붓고, 소금을 넣고 끓입니다.
   - 물이 끓으면 파스타를 넣고 포장지에 적힌 시간에 따라 알 덴테로 삶습니다. (보통 8-10분)
   - 삶은 후 체에 옮겨 물기를 빼고, 다시 팬에 옮겨 놓습니다.

2. **소스 만들기**
   - 팬에 올리브 오일을 약한 불에서 가열합니다.
   - 다진 마늘과 양파를 넣고, 양파가 투명해질 때까지 볶습니다. (약 3-5분)
   - 그 후 다진 토마토를 넣고 잘 섞어줍니다.
   - 소금과 후추로 간을 하고 중불에서 약 10분간 끓입니다. 토마토가 부드러워지고 소스가 농축될 때까지 끓입니다.

3. **파스타와 소스 혼합**
   - 삶은 파스타를 팬에 추가하고, 소스와 잘 섞이도록 저어줍니다. 
   - 필요하다면 파스타의 삶은 물을 조금 더 추가하여 소스를 조절합니다.

4. **마무리 및 서빙**
   - 다 볶아진 파스타에 신선한 바질 잎을 찢어 넣고 잘 섞습니다.
   - 그릇에 담고, 그 위에 파르미지아노 레지아노 치즈를 갈아 올립니다.
   - 원하면 추가적으로 올리브 오일을 약간 뿌려줍니다.

### 팁
- 다양한 야채(버섯, 시금치, 등)를 추가하여 영양을 더욱 풍부하게 만들 수 있습니다.
- 고기를 좋아하신다면, 볶은 소시지나 닭고기를 추가해도 좋습니다.

이 기본 파스타 레시피를 바탕으로 다양한 변형 요리를 시도해 보세요! 즐거운 요리 되시길 바랍니다.


In [33]:
# 번역 chain -> 입력된 내용을 지정한(입력한) 언어로 번역하는 체인.
# prompt template -> model -> output parser 

prompt_template_trans = PromptTemplate(
    template=dedent("""
    # Instruction
    당신은 다국어가능한 숙력된 번역가입니다. 
    요청된 문장을 {language} 로 번역해 주세요.
                    
    # Input data(번역할 문장)
    {content}
                    
    # Output Indicator
    - 번역한 내용만 출력해 주세요.
    """)
)

translate_chain = prompt_template_trans | model | StrOutputParser()


In [34]:
translate_chain.invoke({"content":"안녕하세요", "language":"독일어"})
# 안녕하세요->독일어로 번역한 응답.



'Hallo'

In [35]:
response_kor = translate_chain.invoke({"content":response, "language":"한국어"})

In [36]:
print(response_kor)

## 기본 파스타 레시피

### 재료
1. 스파게티 또는 원하는 파스타 200g
2. 소금 1큰술 (물에 넣기 위함)
3. 올리브 오일 2큰술
4. 마늘 3쪽 (다진 것)
5. 양파 1개 (다진 것)
6. 토마토 400g (토마토 소스 또는 신선한 토마토 다진 것)
7. 바질 잎 몇 장 (선택 사항)
8. 파르미지아노 레지아노 치즈 (간 것, 선택 사항)
9. 후추 (맛을 내기 위함)

### 조리 도구
- 큰 냄비
- 팬
- 나무 숟가락 또는 주걱
- 체

### 조리 방법

1. **파스타 삶기**
   - 큰 냄비에 물을 충분히 붓고, 소금을 넣고 끓입니다.
   - 물이 끓으면 파스타를 넣고 포장지에 적힌 시간에 따라 알 덴테로 삶습니다. (보통 8-10분)
   - 삶은 후 체에 옮겨 물기를 빼고, 다시 팬에 옮겨 놓습니다.

2. **소스 만들기**
   - 팬에 올리브 오일을 약한 불에서 가열합니다.
   - 다진 마늘과 양파를 넣고, 양파가 투명해질 때까지 볶습니다. (약 3-5분)
   - 그 후 다진 토마토를 넣고 잘 섞어줍니다.
   - 소금과 후추로 간을 하고 중불에서 약 10분간 끓입니다. 토마토가 부드러워지고 소스가 농축될 때까지 끓입니다.

3. **파스타와 소스 혼합**
   - 삶은 파스타를 팬에 추가하고, 소스와 잘 섞이도록 저어줍니다.
   - 필요하다면 파스타의 삶은 물을 조금 더 추가하여 소스를 조절합니다.

4. **마무리 및 서빙**
   - 다 볶아진 파스타에 신선한 바질 잎을 찢어 넣고 잘 섞습니다.
   - 그릇에 담고, 그 위에 파르미지아노 레지아노 치즈를 갈아 올립니다.
   - 원하면 추가적으로 올리브 오일을 약간 뿌려줍니다.

### 팁
- 다양한 야채(버섯, 시금치 등)를 추가하여 영양을 더욱 풍부하게 만들 수 있습니다.
- 고기를 좋아하신다면, 볶은 소시지나 닭고기를 추가해도 좋습니다.

이 기본 파스타 레시피를 바탕으로 다양한 변형 요리를 시도해 보세요! 즐거운 요리 되시길 바랍니다.


In [37]:
translate_chain.invoke({"content":response, "language":"일본어"})

'## 基本のパスタレシピ\n\n### 材料\n1. スパゲッティまたはお好みのパスタ 200g\n2. 塩 大さじ1（お湯に入れるため）\n3. オリーブオイル 大さじ2\n4. ニンニク 3片（みじん切り）\n5. 玉ねぎ 1個（みじん切り）\n6. トマト 400g（トマトソースまたは新鮮なトマトのみじん切り）\n7. バジルの葉 数枚（お好みで）\n8. パルミジャーノ・レッジャーノチーズ（おろし、オプション）\n9. 黒胡椒（味付け用）\n\n### 調理器具\n- 大きな鍋\n- フライパン\n- 木製スプーンまたはへら\n- ざる\n\n### 調理方法\n\n1. **パスタを茹でる**\n   - 大きな鍋に十分な水を入れ、塩を加えて沸騰させます。\n   - 水が沸騰したらパスタを入れ、パッケージに書かれた時間に従ってアルデンテになるまで茹でます。（通常8-10分）\n   - 茹でたらざるに移して水気を切り、再びフライパンに移します。\n\n2. **ソースを作る**\n   - フライパンにオリーブオイルを弱火で熱します。\n   - みじん切りのニンニクと玉ねぎを加え、玉ねぎが透明になるまで炒めます。（約3-5分）\n   - その後、みじん切りのトマトを加えてよく混ぜます。\n   - 塩と黒胡椒で味を調え、中火で約10分間煮ます。トマトが柔らかくなり、ソースが濃縮されるまで煮ます。\n\n3. **パスタとソースを混ぜる**\n   - 茹でたパスタをフライパンに加え、ソースとよく混ぜるようにかき混ぜます。\n   - 必要に応じて、パスタの茹で汁を少し加えてソースを調整します。\n\n4. **仕上げと盛り付け**\n   - 炒めたパスタに新鮮なバジルの葉をちぎって加え、よく混ぜます。\n   - お皿に盛り、上にパルミジャーノ・レッジャーノチーズをおろして乗せます。\n   - お好みでオリーブオイルを少し振りかけます。\n\n### ヒント\n- さまざまな野菜（キノコ、ほうれん草など）を加えて栄養をさらに豊富にすることができます。\n- お肉が好きな方は、炒めたソーセージや鶏肉を追加しても良いです。\n\nこの基本のパスタレシピを元に、さまざまなアレンジ料理を試してみてください！楽しい料理を楽しんでください。'

## Chain과 Chain간의 연결

In [38]:
# food_chain ----> translate_chain
## food_chain_prompt: 변수 - food
## translate_chain : 변수 - language, content

# RunnableParallel({"key":Runnable, "key2":Runnable})
# LCEL에서 RunnableParallel => {"key":Runnable, "key2":Runnable} |
chain = {"content":food_chain, 
         "language":RunnableLambda(lambda x : x["language"]) } | translate_chain

In [39]:
# chain = {"content":food_chain, 
#          "language":RunnableLambda(lambda x : x["language"]) } | RunnablePassthrough()
# chain.invoke({"food":"파스타", "language":"영어"})

In [40]:
res = chain.invoke({"food":"파스타", "language":"영어"})
# food -> food_chain, language -> translate_chain
# food_chain 최종결과(레시피) => {"content":레시피} -전달-> translate_chain


In [ ]:
print(res)

## Basic Pasta Recipe

### Ingredients
- Pasta (spaghetti, penne, etc.) 200g
- Olive oil 2 tablespoons
- Minced garlic 3 cloves
- Onion 1 (medium size)
- Cherry tomatoes 200g
- Salt & pepper
- Basil (fresh or dried) to taste
- Parmesan cheese (optional)

### Cooking Tools
- Large pot
- Pan
- Strainer

### Cooking Method

1. **Cooking Pasta:**
   - Boil water in a large pot and add enough salt. (1 tablespoon of salt per liter of water)
   - Add the pasta to the boiling water and cook for the time indicated on the package (usually 8-12 minutes).
   - Drain the cooked pasta using a strainer, and drizzle with olive oil if desired.

2. **Preparing the Sauce:**
   - Heat olive oil in a pan over medium heat.
   - Add minced garlic and chopped onion, and sauté until the onion becomes translucent.
   - Cut the cherry tomatoes in half and add them to the pan, seasoning with salt and pepper. (Sauté for about 5-7 minutes.)

3. **Mixing:**
   - Add the cooked pasta to the pan and stir well to combi

In [ ]:
chain.invoke({"food":"냉면", "language":"몽골어"})

'### Халуун ногооны жор\n\n#### Орц\n* Халуун ногооны будаа (булгасны будаа эсвэл гурилан будаа) 200г\n* Шөлийг (үхрийн шөл эсвэл загасны шөл) 1L\n* Сүү (2 том халбага)\n* Цуу (2 том халбага)\n* Элсэн чихэр (1 том халбага)\n* Үхрийн саримс (1 жижиг халбага)\n* Огурцы (1 ш)\n* Нэрс (эсвэл алим) (1/2 ш)\n* Төмс (2 ш)\n* Нори (цамцлаг)\n* Улаан перец (сонголт)\n* Сансар (сонголт)\n\n#### Шөл бэлтгэх\n1. **Шөл хийх:** Үхрийн 200г болон 1L ус тавагтаа хийж буцалгана. Буцалж эхлэхэд дунд гал дээр 30 минут орчим буцалгана.\n2. **Шөлний амт тохируулах:** Шөл ууж идэж, амт нь гүнзгий байх үед 2 том халбага сүү, 1 жижиг халбага үхрийн саримсыг нэмж дахин буцалгаад, галын тагийг хааж хөргөнө.\n\n#### Будаа бэлтгэх\n1. **Будаа буцалгах:** Тавагт ус буцалгаж, халуун ногооны будааг хийнэ. Савны зааварт заасан буцалгах хугацааг анхаарч үзүүлэхэд тохиромжтой хугацаагаар буцална.\n2. **Будаагаа хүйтэн усаар угаах:** Будаа буцалсны дараа хүйтэн усанд хэд хэдэн удаа угааж, гадаргуу нь жигд чипс мэт байх 

# 사용자 함수를 Chain에 적용하기
함수를 RunnableLambda로 만들거나 Chain으로 정의

## 사용자 함수를 Runnable로 정의 (RunnableLambda)
- 임의의 함수를 Runnable로 정의 할 수있다.
  - chain에 포함할 기능을 함수로 정의할 때 주로 사용. 
- `RunnableLambda(함수)` 사용
  - 함수는 invoke() 메소드를 통해 입력받은 값을 받을 **한개의 파라미터**를 선언해야 한다.

## 사용자 함수를 Chain으로 정의
- Chain 을 구성하는 작업 사이에 추가 작업이 필요할 경우, 중간 결과를 모두 사용해야 하는 경우 함수로 구현한다.
- `@chain` 데코레이터를 사용해 함수에 선언한다.

### Runnable 에 사용할 **사용자 정의 함수** 구문
- 이전 Chain의 출력을 입력 받는 **파라미터를 한개** 선언한다. (첫번째 파라미터)
- `invoke()`로 호출 할때 전달 하는 추가 설정을 입력받는 파라미터를 선언한다.(두번째 파라미터 - Optional)
  - RunnableConfig 타입의 값을 받는데 Dictionary 형식으로 `{"configuable": {"설정이름":"설정값"}}` 형식으로 받는다.
- 만약 함수가 여러개의 인자를 받는 경우 단일 입력을 받아들이고 이를 여러 인수로 풀어내는 래퍼 함수를 작성하여 Runnable로 만든다.
  ```python
  def plus(num1, num2):
      ...

  def wrapper_plus(nums:dict|list):
      return plus(nums['num1'], nums['num2'])
  ```

Lambda 함수를 넣는 게 아니라, 기존에 정의된 함수를 이용하고 싶다면?

In [42]:
from langchain_core.runnables import RunnableLambda

# 기존에 정의된 함수. 이 함수를 Runnable로 정의
def plus(num1, num2, num3):
    return num1 + num2 + num3
# run1.invoke(1, 2, 3) (X)

def wrapper_plus(nums:list):
    return plus(nums[0], nums[1], nums[2])

run1 = RunnableLambda(wrapper_plus)
run1.invoke([1, 2, 3])



6

`invoke()` 함수에는 인자를 최대 두 개만 넣을 수 있다!   
plus는 인자가 세 개 필요한데요ㅠㅠ? -> `plus` 함수를 호출하는 함수를 하나 더 만듭시다!

In [2]:
# 소재를 이용해서 이야기를 생성하는 chain
# 장문을 입력 받아서 요약하는 chain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from textwrap import dedent

load_dotenv()

True

story 모델, summary 모델에 tem를 각각 다르게 줄 수 있다. 

In [17]:
story_model = ChatOpenAI(model_name = "gpt-4o-mini", temperature=1.0)
story_prompt_template = PromptTemplate(
    template=dedent("""
    # Instruction
    당신은 아이들을 위한 이야기를 창작하는 스토리텔러입니다.
    주어진 소재로 잠자리에서 아이들에게 들려줄 재미있는 이야기를 만들어 주세요. 

    # Input Data
    소재: {topic}
                    
    # Output Indicator
    - 이야기는 30문장 이내로 구성해주세요.
    - 이야기를 구어체로 작성해주세요.

""")
)
 
story_chain = story_prompt_template | story_model | StrOutputParser()

In [20]:
# 요약 chain
summary_model = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0.1)
summary_prompt = PromptTemplate(
    template=dedent("""
    # Instruction
    주어진 문장 내용을 3문장으로 요약 해주세요.
    
    # Input Data
    {content}
""")
)

summary_chain = summary_prompt | summary_model | StrOutputParser()

In [21]:
chain = story_chain | summary_chain
chain.invoke({"topic":"호랑이"})

'옛날 깊은 숲속에 무서운 호랑이가 살았지만, 그는 외로움을 느끼고 있었다. 작은 토끼가 호랑이에게 다가가자, 다른 동물들도 호랑이와 친구가 되고 싶어 했고, 호랑이는 그들의 친절로 행복해졌다. 결국 호랑이는 숲속의 영웅이 되어 모든 동물들과 함께 즐거운 시간을 보내며 존경받는 존재가 되었다.'

In [15]:
from langchain_core.runnables import chain

# 사용자 정의 체인 -> 흐름을 원하는 구조로 정의할 수 있다. 
@chain
def custom_chain(topic:str)->dict[str, str]:
    # story_chain과 summary_chain을 원하는 흐름으로 구현.
    # story_chain의 결과와 summary_chain의 결과를 모두 반환하도록 처리.
    story = story_chain.invoke({"topic":topic})
    summary = summary_chain.invoke({"content":story})

    return {"전체이야기":story, "이야기요약":summary}

In [12]:
type(custom_chain) # RunnableLambda가 되었다

langchain_core.runnables.base.RunnableLambda

In [13]:
res = custom_chain.invoke("호랑이")

In [14]:
print(type(res))
print(res.keys())

<class 'dict'>
dict_keys(['전체이야기', '이야기요약'])


# Cache (임시 저장)

- 응답 결과를 저장해서 같은 질문이 들어오면 LLM에 요청하지 않고 저장된 결과를 보여주도록 한다.
    - 처리속도와 비용을 절감할 수 있다.
    - 특히 chatbot같이 비슷한 질문을 하는 경우 유용하다.
- 저장 방식은 `메모리`, `sqlite` 등 다양한 방식을 지원한다.
  
    ```python
    set_llm_cache(Cache객체)
    ```

In [22]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache

# 대화내역을 캐시에 저장 시작
set_llm_cache(InMemoryCache())
model = ChatOpenAI(model_name="gpt-4o-mini")
response = model.invoke("주요 프로그래밍 언어 5개를 소개해줘")

처음 질문할 때: 10초 걸림

In [23]:
print(response.content)

다양한 프로그래밍 언어들이 있지만, 아래의 5개 언어는 특히 널리 사용되며 중요한 역할을 하고 있습니다:

1. **파이썬 (Python)**:
   - 범용 프로그래밍 언어로, 문법이 간결하고 읽기 쉬워 초보자에게 적합합니다.
   - 데이터 분석, 인공지능, 웹 개발, 자동화 등의 분야에서 많이 사용됩니다.
   - 풍부한 라이브러리와 프레임워크(예: NumPy, Pandas, Flask, Django)를 제공합니다.

2. **자바스크립트 (JavaScript)**:
   - 웹 개발의 표준 언어로, 클라이언트 사이드와 서버 사이드 모두에서 사용됩니다.
   - HTML과 CSS와 함께 웹 페이지의 동적 기능을 구현합니다.
   - Node.js와 같은 프레임워크를 통해 서버 개발에서도 활용됩니다.

3. **자바 (Java)**:
   - 객체지향 프로그래밍 언어로, '한 번 작성하면 어디서든 실행된다'는 슬로건을 가지고 있습니다.
   - 기업 애플리케이션, 안드로이드 앱 개발 등에서 널리 사용됩니다.
   - 다양한 플랫폼에서 실행할 수 있도록 설계되어 있습니다.

4. **C# (C Sharp)**:
   - 마이크로소프트에서 개발한 언어로, 주로 .NET 플랫폼에서 사용됩니다.
   - 웹 애플리케이션, 데스크톱 애플리케이션, 게임 개발(특히 Unity에서) 등 다양한 분야에 사용됩니다.
   - 객체지향 프로그래밍의 특징을 가지고 있으며, 강력한 타입 시스템을 제공합니다.

5. **C++**:
   - C 언어를 기반으로 한 객체지향 프로그래밍 언어로, 시스템 소프트웨어, 게임 개발, 성능이 중요한 응용 프로그램에서 많이 사용됩니다.
   - 메모리 관리와 포인터를 직접 다룰 수 있어 고급 프로그래밍 기법을 지원합니다.
   - 성능이 중요시되는 분야에서 여전히 많이 사용되고 있습니다.

각 언어는 고유한 특성과 장점을 가지고 있으므로, 특정 용도나 프로젝트에 따라 적절한 언어를 선택하는 것이 중요합니다.


In [24]:
response = model.invoke("주요 프로그래밍 언어 5개를 소개해줘")

똑같은 질문 -> 바로 답이 나옴. 똑같은 내용 출력

In [25]:
print(response.content)

다양한 프로그래밍 언어들이 있지만, 아래의 5개 언어는 특히 널리 사용되며 중요한 역할을 하고 있습니다:

1. **파이썬 (Python)**:
   - 범용 프로그래밍 언어로, 문법이 간결하고 읽기 쉬워 초보자에게 적합합니다.
   - 데이터 분석, 인공지능, 웹 개발, 자동화 등의 분야에서 많이 사용됩니다.
   - 풍부한 라이브러리와 프레임워크(예: NumPy, Pandas, Flask, Django)를 제공합니다.

2. **자바스크립트 (JavaScript)**:
   - 웹 개발의 표준 언어로, 클라이언트 사이드와 서버 사이드 모두에서 사용됩니다.
   - HTML과 CSS와 함께 웹 페이지의 동적 기능을 구현합니다.
   - Node.js와 같은 프레임워크를 통해 서버 개발에서도 활용됩니다.

3. **자바 (Java)**:
   - 객체지향 프로그래밍 언어로, '한 번 작성하면 어디서든 실행된다'는 슬로건을 가지고 있습니다.
   - 기업 애플리케이션, 안드로이드 앱 개발 등에서 널리 사용됩니다.
   - 다양한 플랫폼에서 실행할 수 있도록 설계되어 있습니다.

4. **C# (C Sharp)**:
   - 마이크로소프트에서 개발한 언어로, 주로 .NET 플랫폼에서 사용됩니다.
   - 웹 애플리케이션, 데스크톱 애플리케이션, 게임 개발(특히 Unity에서) 등 다양한 분야에 사용됩니다.
   - 객체지향 프로그래밍의 특징을 가지고 있으며, 강력한 타입 시스템을 제공합니다.

5. **C++**:
   - C 언어를 기반으로 한 객체지향 프로그래밍 언어로, 시스템 소프트웨어, 게임 개발, 성능이 중요한 응용 프로그램에서 많이 사용됩니다.
   - 메모리 관리와 포인터를 직접 다룰 수 있어 고급 프로그래밍 기법을 지원합니다.
   - 성능이 중요시되는 분야에서 여전히 많이 사용되고 있습니다.

각 언어는 고유한 특성과 장점을 가지고 있으므로, 특정 용도나 프로젝트에 따라 적절한 언어를 선택하는 것이 중요합니다.


커널 다시 시작. 

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache

# 대화내역을 캐시에 저장 시작
# set_llm_cache(InMemoryCache()) # 메모리에 저장

set_llm_cache(SQLiteCache("llm_cache.sqlite"))

model = ChatOpenAI(model_name="gpt-4o-mini")
response = model.invoke("주요 프로그래밍 언어 5개를 소개해줘")

In [4]:
print(response.content)

물론입니다! 여기 주요 프로그래밍 언어 다섯 가지를 소개합니다.

1. **파이썬 (Python)**:
   - 사용 용도: 웹 개발, 데이터 분석, 인공지능, 자동화 등
   - 특징: 간결하고 읽기 쉬운 문법, 방대한 라이브러리 지원, 대규모 커뮤니티.

2. **자바 (Java)**:
   - 사용 용도: 웹 애플리케이션, 모바일 애플리케이션(Android), 기업 소프트웨어 등
   - 특징: 플랫폼 독립성(Write Once, Run Anywhere), 객체 지향 프로그래밍 지원, 안정성과 성능.

3. **자바스크립트 (JavaScript)**:
   - 사용 용도: 웹 개발(프론트엔드 및 백엔드), 서버 애플리케이션, 모바일 앱 등
   - 특징: 클라이언트 측과 서버 측 모두에서 실행 가능, 비동기 프로그래밍 지원, 다양한 프레임워크와 라이브러리.

4. **C++**:
   - 사용 용도: 시스템 소프트웨어, 게임 개발, 고성능 애플리케이션 등
   - 특징: 속도와 성능이 뛰남, 객체 지향 프로그래밍 지원, 메모리 관리에 대한 높은 유연성.

5. **C#**:
   - 사용 용도: 윈도우 애플리케이션, 게임 개발(Unity), 웹 애플리케이션 등
   - 특징: .NET 프레임워크와의 통합, 강력한 타입 시스템, 객체 지향 프로그래밍 지원.

이 언어들은 각각의 강점과 용도가 있으며, 개발자가 필요에 따라 선택하여 사용할 수 있습니다.


# Streaming 방식 응답 처리
- LLM 응답의 완료를 기다리지 않고 실시간으로 생성되는 토큰.청크를 받아서 처리한다. 
- `모델.invoke(input, config)->응답데이터`: 모델의 응답을 기다렸다가 완료되면 한 번에 반환
- `model.stream(input, config)->Iterator`: 모델이 토큰을 생성하면 바로바로 반환한다. 

=> 결과가 나오면 한 번에 쫙 보여주는 게 아니라, 나오는 순서대로 보여준당

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o-mini")
res = model.stream("AI에 대해서 설명해줘.")
print(type(res))

<class 'generator'>


값을 주는 게 아니라, 값을 제공하는 것을 준다.


In [9]:
for token in model.stream("AI에 대해 설명해줘."):
    print(token.content)


AI
(
인
공지
능
)는
 인간
의
 지
능
을
 모
방
하여
 학
습
,
 문제
 해결
,
 의
사
 결정
 등을
 수행
할
 수
 있는
 컴
퓨터
 시스템
이나
 프로그램
을
 의미
합니다
.
 AI
는
 여러
 분야
에
 걸
쳐
 사용
되
며
,
 그
 주요
 목적
은
 특정
 작업
을
 자동
화
하거나
 인간
의
 능
력을
 확
장
하는
 것입니다
.


AI
는
 일반
적으로
 두
 가지
 주요
 범
주
로
 나
눌
 수
 있습니다
:


1
.
 **
약
한
 AI
 (
N
arrow
 AI
)**
:
 특정
 작업
에
 특
화
된
 AI
로
,
 예
를
 들어
 음
성
 인
식
,
 이미지
 인
식
,
 번
역
 서비스
 등
 특정
 문제
를
 해결
하는
 데
 중
점을
 둡
니다
.
 현재
 상
용
화
된
 많은
 AI
 기술
은
 이
 범
주
에
 해당
합니다
.


2
.
 **
강
한
 AI
 (
General
 AI
)**
:
 인간
과
 같은
 수준
의
 범
용
 지
능
을
 갖
춘
 AI
로
,
 다양한
 문제
를
 해결
하고
 인간
과
 유
사
한
 사고
 능
력을
 가지고
 있는
 것을
 목표
로
 합니다
.
 현재
로
서는
 강
한
 AI
는
 이
론
적인
 개
념
에
 가
깝
고
,
 실제
로
 구현
된
 사례
는
 없습니다
.


AI
 기술
의
 핵
심
은
 머신
러
닝
(
기
계
 학
습
)
과
 딥
러
닝
(
심
층
 학
습
)
입니다
.
 머신
러
닝
은
 데이터
에서
 패
턴
을
 학
습
하여
 예
측
 및
 결
정을
 내
리는
 기술
이고
,
 딥
러
닝
은
 신
경
망
을
 사용
하여
 더욱
 복
잡
한
 데이터
 구조
를
 분석
하는
 방법
입니다
.


AI
는
 의료
,
 금융
,
 자
율
주
행
차
,
 고객
 서비스
 등
 다양한
 분야
에서
 활
발
하게
 활용
되고
 있으며
,
 그
 발전
과
 함께
 윤
리
적
,
 사회
적
 문제
도
 함께
 논
의
되고
 있습니다
.

